# OperatorAware
## Urgency detection for emergency calls
mitchellpkt@protonmail.com

OperatorAware evaluates 911 calls in real-time to assess the nature and direness of the emergency being reported.

As the human operator collects information, OperatorAware converts the audio stream to text, and scans for keywords indicating critical situations (e.g. the word "gun" indicating that there is likely a weapon involved). As the call continues, OperatorAware updates the likelihood of a situation involving:
-  Weapons (e.g. inferred from use of words "gun" and "shot" by a caller)
-  Medical emergencies 
-  Vehicle accidents
-  etc

## Example
Caller reports: **"My neighbor shot my leg, It's bleeding everywhere! He didn't mean to discharge the gun"** 

OperatorAware uses the words {"shot", "bleeding", "gun"} to identify that this is a **medical emergency involving a weapon.**

## Motivation
There have been multiple reported instances of emergency operators hanging up on 911 callers during crises. 

For example, one operator who stated "You could deal with it yourself. I’m not gonna deal with this, okay?” before hanging up on a caller that was performing CPR on a shooting victim ([Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2015/07/29/deal-with-it-yourself-911-dispatcher-tells-panicked-caller-with-dying-friend/?utm_term=.eea24de1e5f3)). 

Another 911 dispatcher, infamous for ending a call with "Ain't nobody got time for this," hung up on reports of street racing and armerd robberies (different [Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2018/04/19/911-dispatcher-jailed-houston-woman-hung-up-on-thousands-of-callers/?noredirect=on&utm_term=.b4bb2b6e8f37) article). In this case, later analysis showed that thousands of calls shorter than 20 seconds were attributed to her hanging up. She stated that she hung up because at those times she did not want to talk to anyone ([BBC](http://www.bbc.com/news/world-us-canada-43822504)).

While 911 callers may hang up at any time, in the vast majority of cases, **the 911 dispatcher should not be the party to terminate the call during a crisis.** If an operator is the party to end a call that has been ranked as likely-severe by the OperatorAware, this should trigger quality assurance review. The latter dispatcher mentioned above hung up on thousands of calls over multiple years. With OperatorAware, she would have come under close scrutiny by the end of her first week

## Use case scale
The [Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2018/04/19/911-dispatcher-jailed-houston-woman-hung-up-on-thousands-of-callers/?noredirect=on&utm_term=.d2fbe079869a) notes: *This single consolidated center for 911 calls opened in 2003 and handles millions of calls every year, according to the Chronicle, or 9,000 a day. Two-thirds of those calls aren’t true emergencies*

*The rest involve people in dire need.*

## Changelog
-  v1, local processing using PocketSphinx
-  v2, trying 2 different implementations of Google voice-to-text (Inst 2 works!)
-  v3, cleaning up to the one that works, moving forward


## Import libraries

In [23]:
import os
import sys
from operator_aware_lib.load_audio_from_filename import *
from operator_aware_lib.evaluate_string import *
from operator_aware_lib.return_dictionary import *
from operator_aware_lib.fetch_transcript import *

## Enter name of file to be transcribed

In [2]:
sound_filename = 'fire_short'

## Load in the audio file

In [3]:
### Form filename
str_audio_filename = os.path.join(
     os.getcwd(),
    'Real_911Calls',
    sound_filename + '.flac')

audio_data, audio_config = load_audio_from_filename(str_audio_filename)

## Retrieve the transcription

In [24]:
transcript_directory = os.path.join(
     os.getcwd(),
    'transcriptions')

fetch_transcript(audio_data, audio_config,transcript_directory,qVerbose=1,force_fresh=0,do_not_save=0)



NameError: name 'transcribe_audio_cloud' is not defined

## Has this call been transcribed?

In [ ]:
# Where would/should the transcript file be stored

transcription_filename = os.path.join(
     os.getcwd(),
    'transcriptions',
    sound_filename+'.txt')

does_transcription_exist = os.path.isfile(transcription_filename)

## Process or import the call

In [ ]:
if does_transcription_exist == True:
    with open(transcription_filename,'r') as f_open:
        TranscriptionString = f_open.read()
        
    print('***************************')
    print('Imported data from:' + transcription_filename)
    print('Transcription:')
    print(TranscriptionString)

else:
    # Detect speech in the audio file
    response = client.recognize(config, audio)

    TranscriptionString = ''
    for result in response.results:
        #print('Transcript: {}'.format(result.alternatives[0].transcript))
        TranscriptionString += ' ' + format(result.alternatives[0].transcript)
        
    with open(transcription_filename,'w') as f_open:
        f_open.write(TranscriptionString)
        f_open.close()
    
    print('***************************')
    print('Fresh transcription stored in: '+ transcription_filename)
    print('Transcription:')
    print(TranscriptionString)
        

## Display transcription

In [ ]:
print(TranscriptionString)

## Search transcription for keywords

In [ ]:
HeardWords = []
is_anything_important = 0

for danger_cat in DangerWords.keys():
    ThisCategory = DangerWords[danger_cat]
    for keyword in ThisCategory:
        if keyword in TranscriptionString:
            is_anything_important = 1
            print('Possible ' + DangerDefs[danger_cat] + ': ' + keyword)

## Classify call

In [ ]:
if is_anything_important == 1:
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print('Likely emergency; operator should not terminate')
else:
    print('---------------------')
    print('No emergency detected')